In [94]:
import os
import pandas as pd
import numpy as np
# read,concatenate, split audio
import pydub
from pydub import AudioSegment
from scipy.io import wavfile
import IPython.display as ipd
import matplotlib.pyplot as plt
%matplotlib inline
# shuffle data in dataframe
from sklearn.utils import shuffle
# split train, test data
from sklearn.model_selection import train_test_split
# transform audio to feature vector
import librosa
# report result
from sklearn.metrics import classification_report,confusion_matrix
# svm
from sklearn import svm
# read, save model
import joblib
import pickle
# fast furier transform
from scipy.io import wavfile as wav
from scipy.fftpack import fft
import numpy as np

# path

In [2]:
data_path = '.../emotion_data/'
audio_path = '.../data_train/'
audio_fix_path = '.../audio_fix_error/'

# prepare data

In [ ]:
# # load praise data
# df_praise = pd.ExcelFile(data_path + 'praise_speed_location.xlsx').parse('Sheet1')
# df_praise['target'] = 1
# # load not praise data
# df_not_praise = pd.ExcelFile(data_path + 'not_praise_speed_location.xlsx').parse('Sheet1')
# df_not_praise['target'] = 0

In [ ]:
# # create data
# df_data = pd.concat([df_praise, df_not_praise], axis= 0)
# # shuffle location of data points in dataframe
# df_data  = shuffle(df_data)
# df_data = df_data.reset_index()
# df_data = df_data.drop('index', axis= 1)
# df_data = df_data[['part_name', 'text', 'target']]

In [ ]:
# # save data
# writer = pd.ExcelWriter(data_path + 'audio_data.xlsx', engine= 'xlsxwriter')
# df_data.to_excel(writer, 'Sheet1', index= False)
# writer.save()

# list file in folder

In [3]:
audio_files = os.listdir(audio_path)

In [4]:
len(audio_files)

1661

In [5]:
def check_audio(audio_name):
    if audio_name in audio_files:
        return 1
    else:
        return 0

# load data

In [6]:
df_data = pd.ExcelFile(data_path + 'audio_data.xlsx').parse('Sheet1')

In [7]:
df_data['check'] = df_data.part_name.apply(check_audio)

In [8]:
df_data = df_data[df_data.check == 1]

In [9]:
df_data

,part_name,text,target,check
0,praise_audio_29.wav,very good job,1,1
2,praise_audio_501.wav,good,1,1
4,not_praise_audio_1206.wav,0,0,1
5,praise_audio_39.wav,good,1,1
6,not_praise_audio_642.wav,0,0,1
8,not_praise_audio_793.wav,0,0,1
9,not_praise_audio_1213.wav,0,0,1
12,not_praise_audio_658.wav,0,0,1
13,not_praise_audio_1208.wav,0,0,1
15,praise_audio_453.wav,very good,1,1


In [11]:
def convert_label(text):
    if text in ['very good', 'very good job']:
        return 1
    elif text in ['good', 'good job', 'good work']:
        return 2
    else:
        return 0

In [12]:
df_data['label_lv2'] = df_data.text.apply(convert_label)

In [13]:
df_data.label_lv2.value_counts()

0    1245
2     186
1     175
Name: label_lv2, dtype: int64

In [14]:
# split train, test data
ls_X = df_data.part_name
y = df_data.label_lv2
ls_train, ls_test, y_train, y_test = train_test_split(ls_X, y, test_size=0.33, random_state=42)

# normalize data

In [15]:
def split_audio_part(startMin, startSec, endMin, endSec , audio_name, path = audio_path, output_format = 'wav'):
    # Time to miliseconds
    startTime = startMin*60*1000+startSec*1000
    endTime = endMin*60*1000+endSec*1000
    # Opening file and extracting segment
    song = pydub.AudioSegment.from_wav(path + audio_name)
    extract = song[startTime:endTime]
    # Saving
    #extract.export(audio_part + audio_part_name, format= output_format)
    return extract

In [16]:
def normalize_audio(audio_name):
    output_format = 'wav'
    # load audio name
    #audio_path = '/media/anhnvh/New\ Volume/Kidtopi_data/emotion_data/data_train/'
    audio_part = pydub.AudioSegment.from_wav(audio_path + audio_name)
    sample_rate, audio = wavfile.read(audio_path + audio_name)
    time_audio = float("{:10.2f}".format(len(audio) / sample_rate))
    if (time_audio > 0) and (time_audio <2):
        time_excess = 2.00 - time_audio
        audio_fix_name = 'audio_fix.wav'
        audio_add = split_audio_part(0, 0, 0, time_excess, audio_fix_name, path = audio_fix_path)
        # concatenate audio
        audio_part = audio_part + audio_add
    elif time_audio > 2:
        audio_part = split_audio_part(0, 0, 0, 2, audio_name, path = audio_path)
    elif time_audio == 2:
        audio_part = audio_part
    else:
        pass
    # return save new audio name
    return audio_part.export(audio_path + audio_name, format= output_format)
    #return audio_part

In [18]:
# normalize_audio("praise_audio_11.wav")

# feature engineering

In [28]:
def audio_to_vector(audio_name, d = 1):
    normalize_audio(audio_name)
    y1, sr1 = librosa.load(audio_path + audio_name)
    # result is 2d-array
    b = librosa.feature.mfcc(y1,sr1)
    if b.shape != (20, 87):
        print('{0}: duration of audio error!'.format(audio_name))
    #return b
    if d == 1:
        # transform ad-array to 1d-array
        return np.reshape(b, (1,np.product(b.shape)))[0]
    elif d == 2:
        # transform ad-array to 2d-array
        return b

In [ ]:
## convert 2d-array to 1d array
# a = np.array([[1,2,3], [4,5,6]])
# np.reshape(a, (1,np.product(a.shape)))

In [37]:
audio_to_vector('praise_audio_149.wav', d=2).shape

(20, 87)

In [95]:
rate, data = wav.read(audio_path + 'praise_audio_149.wav')
fft_out = fft(data)

In [97]:
fft_out.shape

(96000, 2)

# SVM

In [22]:
X_train = np.array([audio_to_vector(x) for x in ls_train])
X_train.shape

(1076, 1740)

In [23]:
X_test = np.array([audio_to_vector(x) for x in ls_test])
X_test.shape

(530, 1740)

In [24]:
#c: svm linear kernal
lin_clf = svm.LinearSVC()
lin_clf.fit(X_train, y_train)
all_preds = lin_clf.predict(X_test)
print(classification_report(y_pred = all_preds, y_true= y_test))

              precision    recall  f1-score   support

           0       0.88      0.88      0.88       408
           1       0.49      0.68      0.57        53
           2       0.42      0.30      0.35        69

    accuracy                           0.79       530
   macro avg       0.60      0.62      0.60       530
weighted avg       0.78      0.79      0.78       530



/home/anhnvh/anaconda3/lib/python3.6/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


In [25]:
# d poly kernel
poly_clf = svm.SVC(kernel= 'poly', gamma= 0.1).fit(X_train, y_train)
#poly_clf = svm.SVC(kernel= 'poly').fit(X_train, y_train)
poly_clf.score(X_test, y_test)
all_preds = poly_clf.predict(X_test)
print(classification_report(y_pred = all_preds, y_true= y_test))

              precision    recall  f1-score   support

           0       0.91      0.93      0.92       408
           1       0.65      0.83      0.73        53
           2       0.56      0.36      0.44        69

    accuracy                           0.84       530
   macro avg       0.70      0.71      0.69       530
weighted avg       0.83      0.84      0.84       530



In [26]:
# e rbf kernel
rbf_clf = svm.SVC(kernel='rbf',gamma=0.1).fit(X_train, y_train)
rbf_clf.score(X_test, y_test)
all_preds = rbf_clf.predict(X_test)
print(classification_report(y_pred = all_preds, y_true= y_test))

              precision    recall  f1-score   support

           0       0.77      1.00      0.87       408
           1       1.00      0.02      0.04        53
           2       1.00      0.01      0.03        69

    accuracy                           0.77       530
   macro avg       0.92      0.34      0.31       530
weighted avg       0.83      0.77      0.68       530



# create feature vector for  CNN 
+ (run in colab - GPU)

In [56]:
X_train_CNN = np.array([audio_to_vector(x, d=2) for x in ls_train])

In [57]:
X_test_CNN = np.array([audio_to_vector(x, d=2) for x in ls_test])

In [79]:
# Reshape the data with dimentions of the form: 28x28x1 
# (num_of_samples, weight, height, channel)
x_train = X_train_CNN.reshape(X_train_CNN.shape[0], 20, 87, 1)
x_test = X_test_CNN.reshape(X_test_CNN.shape[0], 20, 87, 1)

In [86]:
joblib.dump(X_train_CNN, 'model/X_train_CNN.pkl')
joblib.dump(X_test_CNN, 'model/X_test_CNN.pkl')

['/home/anhnvh/Desktop/Kidtopi/emotion_classification/model/X_test_CNN.pkl']

In [91]:
joblib.dump(list(y_train), 'model/y_train_CNN.pkl')
joblib.dump(list(y_test), 'model/y_test_CNN.pkl')

['/home/anhnvh/Desktop/Kidtopi/emotion_classification/model/y_test_CNN.pkl']

In [ ]:
#one-hot-encoding
Y_train = np_utils.to_categorical(y_train, 3)
Y_test = np_utils.to_categorical(y_test, 3)

In [ ]:
print('Dữ liệu y ban đầu ', y_train[0])
#print('Dữ liệu y sau one-hot encoding ',y_train_one_hot[0])
print('Dữ liệu y sau one-hot encoding ',Y_train[0])

+ Buil c CNN:
+ Conv 32 (3×3) → ReLU → Conv 64 (3×3) → ReLU → Max pool (2×2)
→ FC 128 → ReLU → Dropout (0.25) → FC 10 → Softmax

In [83]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.utils import np_utils
from keras.datasets import mnist
from tensorflow.python.keras import backend as k

In [84]:
# define model
model = Sequential()

# add convolution layer with 32 kernel have size (3x3), use ReLU function
# chỉ rõ input đầu vào (28, 28, 1)
model.add(Conv2D(32, (3, 3), activation= 'relu', input_shape= (20, 87, 1)))

# add convolution layer with 64 kernel have size (3x3), use ReLU function
model.add(Conv2D(64, (3, 3), activation= 'relu'))

# add Max pooling layer with pool_size = 2x2
model.add(MaxPooling2D(pool_size = (2, 2)))

# add Flatten layer convert from tensor to vector
model.add(Flatten())

# add Fully Connected layer with 128 nodes and use ReLU function
model.add(Dense(128, activation= 'relu'))

# add Dropout(0.25)
model.add(Dropout(0.25))

# Output layer with 10 nodes and use softmax function
model.add(Dense(10, activation= 'softmax'))

# should ouput
model.summary()

RuntimeError: It looks like you are trying to use a version of multi-backend Keras that does not support TensorFlow 2.0. We recommend using `tf.keras`, or alternatively, downgrading to TensorFlow 1.14.

In [ ]:
# compile model, setup loss function and optimizer
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
# train model
# his = model.fit(x_train, Y_train, validation_data= (x_test, Y_test), batch_size= 128, epochs= 30)
his = model.fit(x_train, Y_train, batch_size= 2, epochs= 30)